In [1]:
import datetime
import python as SP
from osgeo import ogr, osr
import os
import numpy as np
import random as rnd
import math
import itertools as iter
import extractFiles as ex
import cars as cc1
import matplotlib.pyplot as plt
import extractDistances as exd

In [2]:
# load the buildings layer "LänmaterietLayer"
InputFileLocation = r'../Data/NewFolder/buildings-epsg3006.shp' # location
shapefile1 = InputFileLocation
buildingDS1 = ogr.Open(shapefile1)
buildingDS2 = ogr.Open(shapefile1)
buildingDS3 = ogr.Open(shapefile1)


# print the number of layers
print("Layer names:", SP.list_of_layers(buildingDS1))


# load the first layer
buildingsLayer = buildingDS1.GetLayer()

# check that the spatial reference is a metric spatial reference
print("spatial reference system: ",buildingsLayer.GetSpatialRef().ExportToWkt())

# filter to remove buildings with area less than 10m2
buildingsLayer.SetAttributeFilter("OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10")
buildingsLayer.ResetReading() # reset counting of the filter

# print the layer fields
print("Layer Fields:", SP.get_layer_fields(buildingsLayer))

# print the tags from the interesting fieldDefn
print("Field tags: ", SP.get_field_tags(buildingsLayer, "ANDAMAL_1"))


## RESIDENTIAL LAYER
# load the layer to residential
ResbuildingsLayer = buildingDS1.GetLayer()
# we are interested in the layer residential codes 133 and 135, 199
ResbuildingsLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (133, 135, 199)"
)
ResbuildingsLayer.ResetReading() # reset counting of the filter
print("Number of residential features: ", ResbuildingsLayer.GetFeatureCount())


## WORKPLACE LAYER
# load the layer to workplace
workplacesLayer = buildingDS2.GetLayer()
# workplaces are 240:299, 302, 304, 307, 311, 319, 322, 499
workplacesLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (240, 242, 243, 247, 248, 249, 252, 253, 299, "
"302, 304, 307, 311, 319, 322, 499)"
)
workplacesLayer.ResetReading() # reset counting of the filter
print("Number of workplace features: ", workplacesLayer.GetFeatureCount())


## OTHERS LAYER
# load the layer to other
otherLayer = buildingDS3.GetLayer()
# Otherplaces (leisure and shopping) are 301, 309, 313, 316, 317, 320, 399, 799
otherLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (301, 309, 313, 316, 317, 320, 399, 799)"
)
otherLayer.ResetReading() # reset counting of the filter
print("Number of other features: ", otherLayer.GetFeatureCount())


# Save the layers
SP.create_buffer_and_projectLayer(ResbuildingsLayer, 3006, 3006, "residentialLayer", bufferDist = 0)
SP.create_buffer_and_projectLayer(workplacesLayer, 3006, 3006, "workLayer", bufferDist = 0)
SP.create_buffer_and_projectLayer(otherLayer, 3006, 3006, "otherLayer", bufferDist = 0)




Layer names: ['buildings-epsg3006']
spatial reference system:  PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]
Layer Fields: ['DETALJTYP', 'ANDAMAL_1', 'ANDAMAL_1T', 'area']
Field tags:  {130, 131, 132, 133, 135, 399, 799, 299, 301, 302, 304, 305, 306, 307, 308, 309, 310, 313, 314, 699, 316, 317, 318, 319, 320, 321, 322, 315, 199, 599, 240, 242, 499, 243, 246, 247, 249, 252, 253}
Number of residential features:  5406
Number of workplace features:  2697
Number of other features:  1391


In [3]:
# Open Parking layer
InputFileLocation = r'../Data/NewFolder/OSM-epsg3600.shp' # location
shapefile2 = InputFileLocation
parkingDB = ogr.Open(shapefile2)
parkingLayer = parkingDB.GetLayer()

# print the parking layer tags
print("Amenity Field tags: ", SP.get_field_tags(parkingLayer, "amenity"))
print("Building Field tags: ", SP.get_field_tags(parkingLayer, "building"))
# Remove None features
parkingLayer.SetAttributeFilter( "NOT OGR_GEOM_WKT LIKE 'None%' AND OGR_GEOM_AREA > 10"
  "AND (amenity IN ('parking', 'parking_space') OR building in ('garage', 'garages'))")
parkingLayer.ResetReading() # reset counting of the filter
print("Number of parking features: ", parkingLayer.GetFeatureCount())


# Open Buffered layer
InputFileLocation = r'UppsalaParkingBuffer100meter3006.shp' # location
shapefile3 = InputFileLocation
parkingBuffer = ogr.Open(shapefile3)
parkingBufferLayer = parkingBuffer.GetLayer()


Amenity Field tags:  {'bicycle_parking', 'cafe', 'car_wash', 'fire_station', 'animal_shelter', 'bar', 'monastery', 'community_centre', 'social_centre', 'events_venue', 'university', 'nursing_home', 'prison', 'kindergarten', 'shelter', 'clinic', 'school', 'vehicle_inspection', 'motorcycle_parking', 'childcare', 'doctors', 'car_rental', 'theatre', 'social_facility', 'parking_space', 'place_of_worship', 'library', 'restaurant', 'parking', 'grave_yard', 'toilets', 'fountain', None, 'boat_storage', 'concert_hall', 'fast_food', 'recycling', 'hospital', 'fuel', 'waste_disposal', 'nightclub', 'animal_training'}
Building Field tags:  {'farm_auxiliary', 'industrial', 'commercial', 'garages', 'greenhouse', 'church', 'yes', 'public', 'barn', 'office', 'silo', 'boathouse', 'university', 'warehouse', 'hut', 'cathedral', 'shed', 'kindergarten', 'school', 'preschool', 'hotel', 'service', 'roof', 'train_station', 'sport', 'house', 'stable', 'hangar', 'terrace', 'residential', 'detached', 'cabin', None,

In [ ]:
# Save figure
fig = plt.figure(figsize = (50,50))
SP.plot_features(parkingLayer, "parkingPlot.pdf", 'k')
SP.plot_features(otherLayer, "parkingPlot.pdf", '#208778')
SP.plot_features(workplacesLayer, "parkingPlot.pdf", '#992266')
SP.plot_features(ResbuildingsLayer, "parkingPlot.pdf", '#1779b2')
parking_patch = mpatches.Patch(color='k', label='Parking lots')
work_patch = mpatches.Patch(color='#992266', label='Workplaces buildings')
other_patch = mpatches.Patch(color='#208778', label='Other buildings')
residential_patch = mpatches.Patch(color='#1779b2', label='Residential buildings')
fntsize= 40
plt.legend(handles= [parking_patch, work_patch, other_patch, residential_patch],
            fontsize = fntsize)
plt.xlim(641, 657)
plt.ylim(6630, 6645)
plt.xlabel("Easting (km)", fontsize = fntsize+20)
plt.ylabel("Northing (km)", fontsize = fntsize+20)
plt.xticks(fontsize = fntsize)
plt.yticks(fontsize = fntsize)
fig.savefig("Map.pdf")

In [4]:
# Calculate intersection of the buffered parking lot layer with the buidlings layers
areaPercentage = SP.get_percentage_of_area_types(parkingBufferLayer,
                [ResbuildingsLayer, workplacesLayer, otherLayer])
np.savetxt("areaPercentage.txt", areaPercentage)

# Get the areas of the parking lots
parkingLotAreas = SP.get_features_areas(parkingLayer)
np.savetxt("parkingLotAreas.txt", parkingLotAreas)


In [5]:
# Get station IDs from the data set
ID = SP.get_field_tags(parkingLayer, "osm_way_id")
# Assert there are no duplicate IDs
assert len(ID) == parkingLayer.GetFeatureCount(), "there are duplicate IDs, create your own IDs for the stations" 


with open('ID.txt','w') as file:
    file.write("\n".join(list(ID)))


In [6]:
stations = SP.create_charging_stations(ID, 
                               parkingLotAreas, 
                               areaPercentage,
                               areaPerCar = 36,     
                               charging_status = True, 
                               charging_power = 3.7)

In [16]:
# Number of state 0 stations
print("number of home stations:",
      len([x for x in stations if x.state ==0]))
# Number of state 1 stations
print("number of work stations:", 
      len([x for x in stations if x.state ==1]))
# Number of state 2 stations
print("number of other stations:", 
      len([x for x in stations if x.state ==2]))

number of home stations: 1350
number of work stations: 979
number of other stations: 733


In [18]:
# Number of parking palaces for state 0 stations
print("number of home charging ports:", 
      sum([x.maximumOccupancy for x in stations if x.state ==0]))
# Number of parking palaces for state 1 stations
print("number of work charging ports:", 
      sum([x.maximumOccupancy for x in stations if x.state ==1]))
# Number of parking palaces for state 2 stations
print("number of other charging ports:", 
      sum([x.maximumOccupancy for x in stations if x.state ==2]))



number of home charging ports: 15643
number of work charging ports: 21342
number of other charging ports: 6872


In [34]:
# Create EVs
numberCars = 100

EVs = [cc1.EV(currentLocation = None, currentState = None) for i in range(numberCars)]
# Randomly allocate the initial location of EVs

[x.inital_conditions(stations,0) for x in EVs]


# print location
list(map(lambda x: x.currentLocation, EVs))

['437723098-0',
 '499626584-0',
 '99723899-0',
 '255487813-0',
 '247520003-0',
 '102550808-0',
 '247519987-0',
 '247520000-0',
 '488528261-0',
 '95806873-0',
 '134643434-0',
 '102550864-0',
 '110879911-0',
 '337225277-0',
 '96131421-0',
 '372815448-0',
 '499816581-0',
 '129038104-0',
 '91252421-0',
 '96065139-0',
 '286087596-0',
 '491540628-0',
 '226093955-0',
 '88820552-0',
 '100134382-0',
 '96131319-0',
 '127458432-0',
 '156990436-0',
 '103538351-0',
 '125203744-0',
 '93657162-0',
 '226093960-0',
 '94699775-0',
 '104145838-0',
 '491543412-0',
 '110364112-0',
 '499047030-0',
 '96821050-0',
 '137212627-0',
 '110739345-0',
 '132433634-0',
 '547335191-0',
 '493521486-0',
 '94699800-0',
 '110678318-0',
 '95953424-0',
 '226093959-0',
 '453509237-0',
 '488528260-0',
 '95953425-0',
 '492108183-0',
 '96689281-0',
 '99192263-0',
 '99059587-0',
 '99860906-0',
 '99059573-0',
 '132513797-0',
 '110739397-0',
 '265010800-0',
 '85469970-0',
 '58227811-0',
 '320313964-0',
 '104145830-0',
 '127000953-

In [10]:
# Weekday
# load transition Matrix
weekdayChain = ex.readMatrixfiles("../TransitionMatrix/*weekday*.txt")
# define the tranistion Matrix
weekday = cc1.Markov(weekdayChain)

# Weekend
# load transition Matrix
weekendChain = ex.readMatrixfiles("../TransitionMatrix/*weekend*.txt")
# define the tranistion Matrix
weekend = cc1.Markov(weekendChain)

In [11]:
# load the weekday distances
weekdayDistances = exd.extractDistances("../distanceData/*day*.txt")

# load the weekend distances
weekendDistances = exd.extractDistances("../distanceData/*end*.txt")


In [35]:
# create a simulation for one month in a year
daysArray = np.arange('2018-01-01', '2018-02-01', dtype='datetime64[D]')
lengthOfDaySimulation = 1440 # 1440 timestep a day

load = np.zeros(shape = (len(daysArray)*lengthOfDaySimulation, len(stations)))

for day in range(len(daysArray)):
    
    if np.is_busday(daysArray[day]):
        chain = weekday
        dist = weekdayDistances
    else:
        chain = weekend
        dist = weekendDistances
    
    # Setup the simulation Model
    simulationCase = cc1.Simulation(stations,
                            EVs,
                            chain,
                            dist)

    # Estimate the electric load
    temp_load = simulationCase.simulate_model(lengthOfDaySimulation)
    
    initialIdx = day*1440
    finalIdx = initialIdx +1440
    load[initialIdx:finalIdx, ::] = temp_load

In [24]:
load = np.loadtxt("load.txt", ndmin = 2)
np.asarray(load).shape

(2880, 3062)

In [25]:
# find the load of the residential load
homeLoad = SP.extract_stateLoad(load, 0, stations)
homeLoad.shape

(7200, 1350)

In [19]:
# collect the load of the stations
final_results = SP.collect_stations_results(ID, load, stations)        
final_results.shape

(7200, 1718)

In [20]:
np.savetxt("finalResults.txt", final_results)

In [36]:
fig = plt.figure(figsize = (50,50))
minutes = np.arange('2018-01-01', '2018-02-01', dtype='datetime64[m]').astype(datetime.datetime)
plt.plot(minutes, np.sum(load,1))
plt.xticks(rotation='vertical')
plt.rcParams.update({'font.size': 25})
fig.savefig("aggregateLoad.pdf")

In [17]:
drvingDist = list(map(lambda x: x.distance/5, EVs))
d = np.asarray(drvingDist)
np.mean(d)

31.286735999999998

In [13]:
import imp
imp

<module 'cars' from '/Users/mahmoudshepero/Documents/PhD/Models/EVSpatialModelPython/PythonCode/cars.py'>